In [2]:
import pandas as pd
import json
from tqdm import tqdm

In [2]:
def process_dialogue(dialog: list):
    dialogue_full = ""
    for utterance in dialog:
        dialogue_full += utterance + " "
    return dialogue_full[:-1]

In [5]:
test = []
with open("test.json", "r") as f:
    for jsonObj in f:
        test.append(json.loads(jsonObj))
        
df = pd.read_csv("CICERO_gen_gold_best.csv")
knowledges = df["knowledge"]

In [9]:
print(len(test))
print(len(knowledges))

10898
14948


In [6]:
checked_dialogue_history = [] #should get 14907 non-empty at the end, 14948 total
targets = []
unchecked = []
unchecked_idx = []
idx = 0
for knowledge in tqdm(knowledges):
    found = False
    for sample in test:
        choices = sample["Correct Answers"]
        for choice in choices:
            checked_knowledge = sample["Choices"][choice] 
            if knowledge[1:-1] == checked_knowledge:
                dh = process_dialogue(sample["Dialogue"])
                target = sample["Target"]
                checked_dialogue_history.append(dh)
                targets.append(target)
                found = True
                break
        if found:
            break
    if not found:
        unchecked.append(knowledge)
        unchecked_idx.append(idx)
        checked_dialogue_history.append("")
        targets.append("")
    idx += 1
            

100%|███████████████████████████████████████████████████████████████████████| 14948/14948 [00:40<00:00, 367.19it/s]


In [7]:
print(f"Final number of dialogue is {len(checked_dialogue_history)}")
count_empty = 0
for dh in checked_dialogue_history:
    if dh == "":
        count_empty += 1
print(f"There are {count_empty} empty dialogues. There should be 41")
print("Same for target.")
        

Final number of dialogue is 14948
There are 41 empty dialogues. There should be 41
Same for target.


In [25]:
checked_dialogue_history

["A: Any messages, Miss Grey? B: Just one, Mr. Blank. You had a telephone call from someone called Brown, David Brown. A: Brown? I don't seem to know anyone called Brown. What did he say? B: He wouldn't say. But it sounded important. I told him you'd phone him as soon as you got back. A: Well, I'd better do it then, I suppose. Er...you've got his phone number, haven't you? B: Yes, it's 633201. A: 622301. B: No, 633201. A: Oh, I'd better write it down, otherwise I'll probably forget it. B: I have already done it, Mr. Blank. It's on your desk.",
 "A: Any messages, Miss Grey? B: Just one, Mr. Blank. You had a telephone call from someone called Brown, David Brown. A: Brown? I don't seem to know anyone called Brown. What did he say? B: He wouldn't say. But it sounded important. I told him you'd phone him as soon as you got back. A: Well, I'd better do it then, I suppose. Er...you've got his phone number, haven't you? B: Yes, it's 633201. A: 622301. B: No, 633201. A: Oh, I'd better write it 

In [12]:
df_best = pd.read_csv("CICERO_gen_gold_best.csv")
df_baseline = pd.read_csv("CICERO_gen_gold_baseline.csv")
knowledge_best = list(df_best["knowledge"])
response_best = list(df_best["response"])
knowledge_baseline = list(df_baseline["knowledge"])
response_baseline = list(df_baseline["response"])

In [13]:
df_best_resp_dh = pd.DataFrame({"response":response_best, "knowledge":checked_dialogue_history})
df_baseline_resp_dh = pd.DataFrame({"response":response_baseline, "knowledge":checked_dialogue_history})
df_gold_dh = pd.DataFrame({"response":knowledge_best, "knowledge":checked_dialogue_history})
df_target_dh = pd.DataFrame({"response":targets, "knowledge":checked_dialogue_history})

In [45]:
df_best_resp_dh.to_csv("CICERO_gen_dh_best.csv")
df_baseline_resp_dh.to_csv("CICERO_gen_dh_baseline.csv")
df_gold_dh.to_csv("CICERO_gold_dh.csv")
df_target_dh.to_csv("CICERO_target_dh.csv")

In [14]:
df_target_dh.to_csv("CICERO_target_dh.csv")

In [14]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

df_best_resp_dh = pd.read_csv("CICERO_gen_dh_best.csv")
df_baseline_resp_dh = pd.read_csv("CICERO_gen_dh_baseline.csv")
df_gold_dh = pd.read_csv("CICERO_gold_dh.csv")
df_target_dh = pd.read_csv("CICERO_target_dh.csv")

In [9]:
if df_best_resp_dh.iloc[121]["knowledge"] is np.nan:
    df_best_resp_dh["knowledge"][121] = "Nothing"

/tmp/ipykernel_31125/1671933020.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_best_resp_dh["knowledge"][121] = "Nothing"


In [10]:
df_best_resp_dh.iloc[121]["knowledge"]

'Nothing'

In [11]:
csvs = [df_best_resp_dh, df_baseline_resp_dh, df_gold_dh, df_target_dh]

In [24]:
for csv in csvs:
    for idx in tqdm(range(len(csv))):
        if csv["response"][idx] is np.nan:
            csv["response"][idx] = "Missing"
        if csv["knowledge"][idx] is np.nan:
            csv["knowledge"][idx] = "Missing"
    

       Unnamed: 0                                           response  \
0               0   Miss grey has received a call from david brow...   
1               1   Mr. blank informed miss grey that he had rece...   
2               2   Mr. blank informed miss grey that he had rece...   
3               3   Miss grey is curious to know about david brow...   
4               4            Mr.blank had a call from david brown.\n   
...           ...                                                ...   
14943       14943   The speaker and the listener decided to give ...   
14944       14944   Mark asked david if he had seen him since sev...   
14945       14945   The speaker and his wife recently purchased a...   
14946       14946   The speaker is not sure about the size of tv ...   
14947       14947   Amy has won the ticket for the olympic openin...   

                                               knowledge  
0      A: Any messages, Miss Grey? B: Just one, Mr. B...  
1      A: Any mes

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 14948/14948 [00:00<00:00, 84689.81it/s]


       Unnamed: 0                                           response  \
0               0      Miss grey has received a call from mr. brown.   
1               1   Mr. Blank told mrs. grey that he had received...   
2               2   Mr. Blank told mrs. grey that he had received...   
3               3    Miss grey is curious to know about david brown.   
4               4               Mr. Blank had a call with mrs. grey.   
...           ...                                                ...   
14943       14943   The speaker is trying to convince the kids to...   
14944       14944   Mark's friend david dropped out of his school...   
14945       14945   The speaker and his family have moved to a ne...   
14946       14946   The speaker and the listener are planning to ...   
14947       14947   Amy and the speaker are friends since childhood.   

                                               knowledge  
0      A: Any messages, Miss Grey? B: Just one, Mr. B...  
1      A: Any mes

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 14948/14948 [00:00<00:00, 99809.69it/s]


       Unnamed: 0                                           response  \
0               0   Miss grey received a phone call from david br...   
1               1   The listener tries to recall if he knows mr. ...   
2               2   The listener tries to recall if he knows mr. ...   
3               3   Mr blank is curious to know what message mr d...   
4               4   David brown has an important work with mr bla...   
...           ...                                                ...   
14943       14943   The speaker and the listener are discussing h...   
14944       14944   The speaker informs mark that their friend da...   
14945       14945   The speaker's house was in the local governin...   
14946       14946   The speaker has enough money to purchase a bi...   
14947       14947   Amy was upset yesterday as she could not get ...   

                                               knowledge  
0      A: Any messages, Miss Grey? B: Just one, Mr. B...  
1      A: Any mes

100%|████████████████████████████████████████████████████████████████████████████████████████████| 14948/14948 [00:00<00:00, 100104.83it/s]


       Unnamed: 0                        response  \
0               0  Hi Amy , you look very happy .   
1               1  Hi Amy , you look very happy .   
2               2  Hi Amy , you look very happy .   
3               3  Hi Amy , you look very happy .   
4               4  Hi Amy , you look very happy .   
...           ...                             ...   
14943       14943  Hi Amy , you look very happy .   
14944       14944  Hi Amy , you look very happy .   
14945       14945  Hi Amy , you look very happy .   
14946       14946  Hi Amy , you look very happy .   
14947       14947  Hi Amy , you look very happy .   

                                               knowledge  
0      A: Any messages, Miss Grey? B: Just one, Mr. B...  
1      A: Any messages, Miss Grey? B: Just one, Mr. B...  
2      A: Any messages, Miss Grey? B: Just one, Mr. B...  
3      A: Any messages, Miss Grey? B: Just one, Mr. B...  
4      A: Any messages, Miss Grey? B: Just one, Mr. B...  
...      

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 14948/14948 [00:00<00:00, 99874.72it/s]


In [22]:
df_best_resp_dh.iloc[121]

Unnamed: 0                                                  121
response       The listener checks the seat number of the sp...
knowledge                                                   NaN
Name: 121, dtype: object

In [21]:
df_best_resp_dh["knowledge"][121] is np.nan

True

In [23]:
csvs[0]["knowledge"][121]

'Nothing'

In [25]:
csvs[0].to_csv("CICERO_gen_dh_best.csv")
csvs[1].to_csv("CICERO_gen_dh_baseline.csv")
csvs[2].to_csv("CICERO_gold_dh.csv")
csvs[3].to_csv("CICERO_target_dh.csv")

In [3]:
df_target_dh = pd.read_csv("CICERO_target_dh.csv")

In [5]:
df_target_dh.iloc[121]

Unnamed: 0    121
response      NaN
knowledge     NaN
Name: 121, dtype: object

In [7]:
import numpy as np
for idx in tqdm(range(len(df_target_dh))):
    if df_target_dh["response"][idx] is np.nan:
        df_target_dh["response"][idx] = "Missing"
    if df_target_dh["knowledge"][idx] is np.nan:
        df_target_dh["knowledge"][idx] = "Missing"

  0%|                                                                                    | 0/14948 [00:00<?, ?it/s]/tmp/ipykernel_28398/1922248964.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target_dh["response"][idx] = "Missing"
/tmp/ipykernel_28398/1922248964.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target_dh["knowledge"][idx] = "Missing"
100%|█████████████████████████████████████████████████████████████████████| 14948/14948 [00:00<00:00, 67608.48it/s]


In [9]:
df_target_dh.to_csv("CICERO_target_dh.csv")